In [1]:
import os
import re
import sys
import nltk
import time
import string
#sys.path.append("..")

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
proj_path = os.path.dirname(os.path.realpath('url_extraction.ipynb'))
data_path = os.path.join(proj_path, 'data/')
results_path = os.path.join(proj_path, 'results/')
model_path = os.path.join(proj_path, 'models/')

train_path=os.path.join(data_path, 'train_v2.csv')
test_path=os.path.join(data_path, 'test_v2.csv')
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
train_labels = train_data['category'].tolist()

In [3]:
#train_data = train_data.iloc[4913:,:].copy()
#test_data = test_data.iloc[2441:,:].copy()
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'}

In [4]:
#train_data = train_data.tail(5)
#test_data = test_data.tail(5)

In [ ]:
#Process Training Data

url_text = list()
url_status = list()
processed_titles = list()
processed_texts = list()
year_of_publish = list()
total_rows = len(train_data.index)
row_num = 0
for row in train_data.iterrows():
    row_num += 1
    try:
        page = requests.get(row[1]['url'], headers=headers)
        pagetext = page.text
        pagecode = page.status_code
    except:
        pagetext = ""
        pagecode = 1

    soup = BeautifulSoup(pagetext, 'html.parser')
    for script in soup(["script", "style"]):
        script.decompose()
    raw_text = soup.get_text()

    porter = nltk.PorterStemmer()
    tokens = nltk.wordpunct_tokenize(raw_text)
    stop = nltk.corpus.stopwords.words('english') + list(string.punctuation)
    tokens = [i.lower() for i in tokens if i not in stop]
    tokens = [porter.stem(t) for t in tokens]
    text = nltk.Text(tokens)
    if pagecode == 200:
        processed_text = ' '.join(tokens)
    else:
        processed_text = ''

    porter = nltk.PorterStemmer()
    tokens = nltk.wordpunct_tokenize(row[1]['title'])
    stop = nltk.corpus.stopwords.words('english') + list(string.punctuation)
    tokens = [i.lower() for i in tokens if i not in stop]
    tokens = [porter.stem(t) for t in tokens]
    text = nltk.Text(tokens)
    processed_title = ' '.join(tokens)
    
    datetime = time.gmtime(row[1]['timestamp']/1000.)
    year = datetime.tm_year
    year_of_publish.append(year)
    
    
    url_text.append(raw_text)
    url_status.append(pagecode)
    processed_texts.append(processed_text)
    processed_titles.append(processed_title)
    
    temp_df = train_data.head(row_num).copy()
    temp_df['url_status'] = url_status    
    temp_df['url_text'] = url_text
    temp_df['processed_titles'] = processed_titles
    temp_df['processed_texts'] = processed_texts
    temp_df['year_of_publish'] = year_of_publish
    
    col_names = ['article_id','publisher','year_of_publish','processed_titles','url_status','processed_texts','category']
    temp_df = temp_df[col_names]
    temp_df.to_csv('data/temp.csv',index=False)
    
    print ('{} of {} Processed: '.format(row_num,total_rows) + row[1]['url'])

train_data['url_status'] = url_status    
train_data['url_text'] = url_text
train_data['processed_titles'] = processed_titles
train_data['processed_texts'] = processed_texts
train_data['year_of_publish'] = year_of_publish

col_names = ['article_id','publisher','year_of_publish','processed_titles','url_status','processed_texts','category']
train_data = train_data[col_names]
train_data.to_csv('data/train_processed.csv',index=False)

train_data.head()

In [5]:
#Process Test Data

url_text = list()
url_status = list()
processed_titles = list()
processed_texts = list()
year_of_publish = list()
total_rows = len(test_data.index)
row_num = 0

for row in test_data.iterrows():
    row_num += 1
    try:
        page = requests.get(row[1]['url'], headers=headers)
        pagetext = page.text
        pagecode = page.status_code
    except:
        pagetext = ""
        pagecode = 1

    soup = BeautifulSoup(pagetext, 'html.parser')
    for script in soup(["script", "style"]):
        script.decompose()
    raw_text = soup.get_text()

    porter = nltk.PorterStemmer()
    tokens = nltk.wordpunct_tokenize(raw_text)
    stop = nltk.corpus.stopwords.words('english') + list(string.punctuation)
    tokens = [i.lower() for i in tokens if i not in stop]
    tokens = [porter.stem(t) for t in tokens]
    text = nltk.Text(tokens)
    if pagecode == 200:
        processed_text = ' '.join(tokens)
    else:
        processed_text = ''

    porter = nltk.PorterStemmer()
    tokens = nltk.wordpunct_tokenize(row[1]['title'])
    stop = nltk.corpus.stopwords.words('english') + list(string.punctuation)
    tokens = [i.lower() for i in tokens if i not in stop]
    tokens = [porter.stem(t) for t in tokens]
    text = nltk.Text(tokens)
    processed_title = ' '.join(tokens)
    
    datetime = time.gmtime(row[1]['timestamp']/1000.)
    year = datetime.tm_year
    year_of_publish.append(year)
    
    
    url_text.append(raw_text)
    url_status.append(pagecode)
    processed_texts.append(processed_text)
    processed_titles.append(processed_title)
    
    temp_df = test_data.head(row_num).copy()
    temp_df['url_status'] = url_status    
    temp_df['url_text'] = url_text
    temp_df['processed_titles'] = processed_titles
    temp_df['processed_texts'] = processed_texts
    temp_df['year_of_publish'] = year_of_publish
    
    col_names = ['article_id','publisher','year_of_publish','processed_titles','url_status','processed_texts']
    temp_df = temp_df[col_names]
    temp_df.to_csv('data/temp.csv',index=False)
    
    print ('{} of {} Processed: '.format(row_num,total_rows) + row[1]['url'])

test_data['url_status'] = url_status    
test_data['url_text'] = url_text
test_data['processed_titles'] = processed_titles
test_data['processed_texts'] = processed_texts
test_data['year_of_publish'] = year_of_publish

col_names = ['article_id','publisher','year_of_publish','processed_titles','url_status','processed_texts']
test_data = test_data[col_names]
test_data.to_csv('data/test_processed.csv',index=False)

test_data.head()

1 of 1385 Processed: http://www.hirunews.lk/79743/visa-mastercard-block-transactions-for-sanctioned-banks
2 of 1385 Processed: http://in.reuters.com/article/2014/03/20/italy-factors-march-idINL6N0MB2CE20140320
3 of 1385 Processed: http://in.reuters.com/article/2014/03/18/germany-zew-idINF9N0D300W20140318
4 of 1385 Processed: http://vator.tv/news/2014-03-20-airbnb-seeking-funding-at-reported-10b-valuation
5 of 1385 Processed: http://www.globalpost.com/dispatch/news/afp/140322/credit-suisse-pay-885-mn-over-us-mortgage-bonds-0
6 of 1385 Processed: http://www.huffingtonpost.com/2014/03/19/california-water-restrictions-drought_n_4990132.html
7 of 1385 Processed: http://wegotthiscovered.com/gaming/walmart-entering-games-business/
8 of 1385 Processed: http://www.thestreet.com/story/12534272/1/why-oracle-corporation-orcl-is-tumbling-after-hours.html
9 of 1385 Processed: http://www.investing.com/analysis/pound-trading-slightly-lower-ahead-of-the-boe%E2%80%99s-most-recent-policy-meeti-206550
10 

72 of 1385 Processed: http://www.myfashionlife.com/archives/2014/03/20/mulberry-ceo-bruno-guillon-resigns/
73 of 1385 Processed: http://www.oregonlive.com/business/index.ssf/2014/03/airbnb_said_to_seek_new_financ.html
74 of 1385 Processed: http://www.bostonglobe.com/business/2014/03/20/airbnb-said-pursue-valuation-above/nEJ10NkDQCrmQLBO3QKyXJ/story.html
75 of 1385 Processed: http://www.zdnet.com/icahns-latest-tip-ebay-should-sell-20-percent-of-paypal-via-ipo-7000027489/
76 of 1385 Processed: http://globalnews.ca/news/1223521/harper-imposes-further-sanctions-against-russian-officials-bank-rossiya/
77 of 1385 Processed: http://www.kjrh.com/money/business-news/walmart-to-accept-video-game-tradeins-in-stores_83607422_
78 of 1385 Processed: http://www.clickondetroit.com/lifestyle/health/health-care-reform/affordable-care-act-some-may-have-more-time-to-apply/25094376
79 of 1385 Processed: http://www.deadline.com/2014/03/state-attorneys-general-plan-to-scrutinize-comcast-time-warner-cable-dea

140 of 1385 Processed: http://www.gamerevolution.com/news/walmart-used-preowned-video-games-business-24727
141 of 1385 Processed: http://www.channelpartnersonline.com/news/2014/03/symantec-fires-ceo-steve-bennett-appoints-interim-replacement.aspx
142 of 1385 Processed: http://www.spokesman.com/stories/2014/mar/22/wal-mart-tool-gives-competitors-prices/
143 of 1385 Processed: http://www.washingtonpost.com/business/economy/29-of-30-big-banks-pass-feds-stress-test/2014/03/20/41053092-b06b-11e3-95e8-39bef8e9a48b_story.html\?tid=pm_pop
144 of 1385 Processed: http://www.investing.com/news/commodities-news/gold-drops-on-putin-comments,-u.s.-housing-data-272788
145 of 1385 Processed: http://www.independent.co.uk/news/uk/home-news/quids-in-how-to-spot-if-youve-got-a-fake-one-pound-coin-9201520.html
146 of 1385 Processed: http://www.geekwire.com/2014/airbnb-may-worth-expedia/
147 of 1385 Processed: http://www.examiner.com/article/wal-mart-to-accept-game-trade-ins-store-credit-can-apply-to-non-ga

208 of 1385 Processed: http://www.entrepreneur.com/article/232382
209 of 1385 Processed: http://www.journalgazette.net/article/20140321/BIZ/303219937/1031/BIZ
210 of 1385 Processed: http://washington.cbslocal.com/2014/03/21/us-appeals-court-upholds-fed-cap-on-card-swipe-fees/
211 of 1385 Processed: http://azstarnet.com/tax-time-phone-scam-more-active-than-ever/article_71218ccd-707b-530e-9af5-85fcb1f034d7.html
212 of 1385 Processed: http://www.kcci.com/project-economy/price-comparison-tool-lets-you-compare-80000-products/25091722
213 of 1385 Processed: http://www.mynews13.com/content/news/cfnews13/news/article.html/content/news/articles/cfn/2014/3/16/New_unemployment_numbers_for_Fla_due_out.html
214 of 1385 Processed: http://www.newsmax.com/newsfront/obamacare-healthcare-premiums-insurance/2014/03/19/id/560460/
215 of 1385 Processed: http://sundial.csun.edu/2014/03/merchants-users-of-bitcoin-uncertain-on-future-of-the-cryptocurrency/
216 of 1385 Processed: http://stream.wsj.com/story/la

277 of 1385 Processed: http://www.sltrib.com/sltrib/money/57712439-79/russia-bank-sanctions-russian.html.csp
278 of 1385 Processed: http://www.elleuk.com/fashion/news/mulberry-announces-that-ceo-bruno-guillon-is-stepping-down
279 of 1385 Processed: http://www.baltictimes.com/news/articles/34603/
280 of 1385 Processed: http://www.huffingtonpost.co.uk/2014/03/20/fox-news-noahs-ark-missing-malaysia-plane_n_4999181.html\?utm_hp_ref=uk
281 of 1385 Processed: http://www.rferl.org/content/twitter-turkey-ban-gul-erdogan-media/25304901.html
282 of 1385 Processed: http://www.zdnet.com/symantec-ousts-ceo-bennett-7000027542/
283 of 1385 Processed: http://www.tampabay.com/news/business/workinglife/floridas-unemployment-rate-falls-to-61-percent/2170538
284 of 1385 Processed: http://times-journal.com/news/article_5098dbb6-ae14-11e3-bfc1-0019bb2963f4.html
285 of 1385 Processed: http://www.rttnews.com/2289611/symantec-corp-symc-is-falling-after-ceo-fired.aspx
286 of 1385 Processed: http://www.stltoday.

...

1162 of 1385 Processed: http://o.canada.com/technology/techbiz/mt-gox-bitcoins-found-415223/
1163 of 1385 Processed: http://www.capebretonpost.com/Business/2014-03-21/article-3658463/Nearly-all-major-US-banks-pass-annual-Fed-stress-tests/1
1164 of 1385 Processed: http://finance.yahoo.com/blogs/breakout/today-s-trending-ticker--gamestop--gme-161016209.html
1165 of 1385 Processed: http://www.bloombergview.com/articles/2014-03-21/fed-thinks-big-banks-will-get-bigger-in-a-crisis
1166 of 1385 Processed: http://triblive.com/business/headlines/5809434-74/prices-offering-shoppers
1167 of 1385 Processed: http://www.gainesville.com/article/20140317/WIRE/140319623/-1/entertainment\?Title=Florida-unemployment-rate-dropped-to-6-1-percent-in-January
1168 of 1385 Processed: http://www.thedailystar.net/online/turks-seek-to-challenge-twitter-ban-16630
1169 of 1385 Processed: http://money.msn.com/business-news/article.aspx\?feed=OBR&amp;date=20140321&amp;id=17454402
1170 of 1385 Processed: http://dailyc

1233 of 1385 Processed: http://time.com/33218/russia-crimea-ukraine-sanctions-stocks-putin/\?utm_source=feedburner&amp;utm_medium=feed&amp;utm_campaign=Feed:timeblogs/the_china_blog(TIME:TheChinaBlog)
1234 of 1385 Processed: http://www.dailytech.com/US+States+to+Help+DOJ+Assess+Comcasts+TWC+Acquisition/article34556.htm
1235 of 1385 Processed: http://www.clickondetroit.com/lifestyle/health/health-care-reform/affordable-care-act-some-may-have-more-time-to-apply/25094376
1236 of 1385 Processed: http://vtdigger.org/2014/03/17/interactive-data-vermonts-unemployment-rate-drops-4-percent/
1237 of 1385 Processed: http://www.tampabay.com/news/business/workinglife/floridas-unemployment-rate-falls-to-61-percent/2170538
1238 of 1385 Processed: http://247wallst.com/autos/2014/03/19/toyota-justice-dept-reach-1-billion-settlement-cnn/
1239 of 1385 Processed: http://www.staceypageonline.com/2014/03/17/indiana-unemployment-rate-decreases/
1240 of 1385 Processed: http://www.gmanetwork.com/news/story/353

1300 of 1385 Processed: http://www.sfgate.com/business/bloomberg/article/Fed-Stress-Test-Shows-29-of-30-Banks-Meet-or-Top-5335500.php
1301 of 1385 Processed: http://invezz.com/news/commodities/9795-silver-price-plunges-to-5-week-low-after-fed-rate-hike-surprise
1302 of 1385 Processed: http://www.webpronews.com/malaysian-airlines-french-satellite-spots-possible-debris-2014-03
1303 of 1385 Processed: http://wlfi.com/2014/03/17/local-unemployment-rose-despite-state-wide-decline/
1304 of 1385 Processed: http://www.virtual-strategy.com/2014/03/23/gains-mortgage-backed-securities-may-translate-slightly-better-rates-start-next-week
1305 of 1385 Processed: http://www.bloomberg.com/news/2014-03-19/ebay-applauds-icahn-backing-away-from-push-to-sell-paypal.html
1306 of 1385 Processed: http://www.businesstimes.com.sg/breaking-news/technology/ousted-symantec-ceo-may-get-us185m-severance-payment-20140322
1307 of 1385 Processed: http://online.wsj.com/article/BT-CO-20140319-702597.html
1308 of 1385 Pr

1368 of 1385 Processed: http://www.newschanneldaily.com/walmart-looking-dominate-another-field/3063/richard-schirmer/
1369 of 1385 Processed: http://www.binarytribune.com/2014/03/18/forex-market-eursek-touches-two-week-lows-following-downbeat-ez-economic-sentiment-data/
1370 of 1385 Processed: http://www.globalpost.com/dispatch/news/thomson-reuters/140319/bmw-sees-2014-pretax-profit-single-double-digit-pct
1371 of 1385 Processed: http://247wallst.com/investing/2014/03/22/walmart-offers-data-on-what-competitors-charge/
1372 of 1385 Processed: http://www.businessweek.com/news/2014-03-20/u-dot-k-dot-stocks-drop-after-yellen-rate-hike-comments-as-glaxo-slips
1373 of 1385 Processed: http://www.belfasttelegraph.co.uk/news/local-national/uk/new-1-coin-designed-to-beat-fakers-30105200.html
1374 of 1385 Processed: http://247wallst.com/autos/2014/03/19/toyota-justice-dept-reach-1-billion-settlement-cnn/
1375 of 1385 Processed: http://www.heraldtribune.com/article/20140317/ARTICLE/140319688
1376 

,article_id,publisher,year_of_publish,processed_titles,url_status,processed_texts
2441,2442,Hiru News,2014,visa mastercard block transact sanction bank,200,hirunika premachandra continu spread drug mena...
2442,2443,Reuters,2014,itali factor watch march 20,200,itali factor watch march 20 reuter discov thom...
2443,2444,Reuters,2014,ukrain crisi worri hammer german investor moral,200,ukrain crisi worri hammer german investor mora...
2444,2445,VatorNews,2014,airbnb seek fund report 10b valuat,200,airbnb seek fund report 10b valuat vatornew va...
2445,2446,GlobalPost,2014,credit suiss pay 885 mn us mortgag bond,404,
